In [ ]:
#note: error in the pixel area and flux conversion factor are currently set as zero. 
#To change these alter the values of Pixel_area_err and flux_conv_err respectivley.

In [ ]:
#We define functions for calculating the blackbox style uncertanty in kpc/arcmin and comoving distance

def Cosmo_error(val,err):
    return(
        (abs(((1/cosmo.kpc_comoving_per_arcmin(val+err))**2)-((1/cosmo.kpc_comoving_per_arcmin(val-err))**2))/2).value
          )

def Flux_error(val_1,err_1,val_2,err_2):
    return(
        ((val_1*err_2)**2+(val_2*err_1)**2)**(1/2)
            )

def Comov_error(val,err):
    return(
        abs(((cosmo.comoving_distance(val+err).cgs)-(cosmo.comoving_distance(val-err).cgs))/2).value          
                )

def Luminosity_error(flux,flux_err,dist,dist_err):  
    return(
        ((4*np.pi*(dist**2)*flux_err)**2+(8*np.pi*dist*flux*dist_err)**2)**(1/2)
    )

def Sbrightness_error(lum,lum_err,conv,conv_err):
    return(
        ((conv*lum_err/Pixel_area)**2+(lum*conv_err/Pixel_area)**2+
         (lum*conv*Pixel_area_err/(Pixel_area)**2)**2)**(1/2)
    )

In [ ]:
import numpy as np
from astropy.cosmology import WMAP9 as cosmo
from astropy.table import Table, Column
import astropy.io.ascii as asc

In [ ]:
#This cell defines file paths and initial parameters:

Input_file='Galex flux single.csv'
Sn_name = 0 #column number of SN names from Redshift_file
Red_val = 1 #column number of redshift values from Redshift_file
Sn_val = 2 #column number of counts per second from Data_file
Sn_exp = 3 #column number of exposure time from Data_file

out = Table() #A table for storing information to be written to the output file
Output_file="output.csv"

#Initial paramaters are defined

flux_conv = 2.06*10**(-16) # A pixel count to flux conversion factor in units of (erg cm-2 Å-1)/(N/s)
        #NUV: Flux = 2.06*10**(-16) x CPS, FUV: Flux = 1.40*10**(-15) x CPS  -- Reference: http://asd.gsfc.nasa.gov/archive/galex/FAQ/counts_background.html
flux_conv_err = 0 #error in flux_conv
    
Pixel_area=(1.5/60)**2 #Arcmin per pixel
        #Pixel width is given in arcseconds, converted to arcmin, and squared to find area in arcmin^2.
Pixel_area_err = 0

In [ ]:
data = Table(asc.read(Input_file, data_start=1)) 

In [ ]:
x=((1+300/(299792.458))/(1-300/(299792.458)))**(1/2)-1

#This cell adds the Supernova names to the table "out"
out['Sn Name'] = [row[Sn_name] for row in data]

out['Redshift'] = [float((row[Red_val])) for row in data]

out['Redshift error'] = [((1/1000)**2+(x)**2)**(1/2) for row in out] 
    #Error in redshift taken as 1 in 1000 with an additional 300 km/s contribution from peculiar velocity

out['ArcMin^2 per Kpc^2 at Redshift'] =[
    (1/cosmo.kpc_comoving_per_arcmin(row[1]).value)**2 for row in out]

out['ArcMin^2 / Kpc^2 error'] = [Cosmo_error(row[1],row[2]) for row in out]

out['Pixel Value N/s'] = [float(row[Sn_val]) for row in data]

out['Exposure time (s)'] = [float((row[Sn_exp])) for row in data]

out['Pixel Error N^(1/2)'] = [(row[5]*row[6])**(1/2) for row in out]

out['Flux (erg s-1 cm-2 A-1 px-1)'] = [flux_conv*row[5] for row in out]

out['Flux error (erg s-1 cm-2 A-1 px-1)'] = [Flux_error(row[5],row[7],flux_conv,flux_conv_err) for row in out]

out['Comoving Distance at Redshift (cm)'] = [cosmo.comoving_distance(row[1]).cgs.value for row in out]

out['Comoving Error (cm)'] = [Comov_error(row[1],row[2]) for row in out]

out['Luminosity (erg s-1 A-1 px-1)'] = [row[8]*4*np.pi*row[10]**2 for row in out]

out['Luminosity Error (erg s-1 A-1 px-1)'] = [Luminosity_error(row[8],row[9],row[10],row[11]) for row in out]

out['Surface Brightness (erg s-1 A-1 Kpc^-2)'] = [row[12]/Pixel_area*row[3] for row in out]

out['Surface Brightness error (erg s-1 A-1 Kpc^-2)'] = [Sbrightness_error(row[12],row[13],row[3],row[4]) for row in out]

out



In [ ]:
asc.write(out, Output_file, delimiter=",")
print('done')